In [101]:
import os

In [181]:
# 设置全局字体为支持中文的字体
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei']  # 或者 'SimHei'
plt.rcParams['axes.unicode_minus'] = False  # 正常显示负号

In [103]:
# 读取xyz数据
root = r'C:\Users\81004\Desktop\ptr2\xyz'
file_name = "4000-cry_300k_out.xyz"
#########
file_path = os.path.join(root,file_name)

In [105]:
ana_frame = 0  # 替换为你想要的帧数

In [107]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
# 阿伏伽德罗常数 (单位：mol^-1)
Avogadro_number = 6.022e23
# 元素的相对原子质量 (单位：g/mol)
element_masses = {
    'O': 15.999, 'Si': 28.085, 'Na': 22.990, 'Al': 26.982, 'Zn': 65.38, 
    'F': 18.998, 'Br': 79.904, 'K': 39.098
}
# 读取XYZ文件的函数
def read_xyz(file_path, ana_number):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    num_atoms = int(lines[0].strip())  # 获取原子数
    len_page = num_atoms + 2  # 每一帧的总行数，包括原子数和描述行
    atoms = []
    positions = []
    # 找到最大的xyz
    a,b,c = -1000,-1000,-1000
    # 从指定帧的第3行开始读取元素和坐标
    for line in lines[ana_number * len_page + 2:(ana_number + 1) * len_page]:
        parts = line.split()
        element = parts[0]
        x, y, z = map(float, parts[1:])
        if x>a:
            a = x
        if y>b:
            b = y
        if z>c:
            c = z
        atoms.append(element)
        positions.append([x, y, z])
    box_len = [a+0.1,b+0.1,c+0.1]
    # print(f'Boxsize: {box_len}')
    return np.array(positions), atoms, num_atoms,box_len


In [144]:
def compute_3d_density(positions, elements, box_size, bins=40):
    element_masses = {
        'O': 15.999, 'Si': 28.085, 'Na': 22.990, 'Al': 26.982,
        'Zn': 65.38, 'F': 18.998, 'Br': 79.904, 'K': 39.098
    }

    # 分别计算每个方向的 voxel size
    if isinstance(bins, int):
        bins = [bins, bins, bins]
    voxel_size = box_size / np.array(bins)
    voxel_volume = np.prod(voxel_size)  # voxel体积 (Å³)

    # 初始化体密度网格
    density_map = np.zeros((bins[0], bins[1], bins[2]))

    for pos, elem in zip(positions, elements):
        if elem not in element_masses:
            continue  # 忽略未知元素
        x, y, z = pos

        ix = min(int(x / voxel_size[0]), bins[0] - 1)
        iy = min(int(y / voxel_size[1]), bins[1] - 1)
        iz = min(int(z / voxel_size[2]), bins[2] - 1)

        mass = element_masses[elem]
        density_map[ix, iy, iz] += mass / voxel_volume

    # 单位换算：Å³ → cm³
    density_map *= 1e24
    return density_map


In [446]:
boxsize = np.array([81.3982, 81.3982, 81.3982]) 

In [450]:
import numpy as np
import matplotlib.pyplot as plt

def plot_xy_density_in_z_range(density_map, box_size, z_range, bin_xy, method='sum'):
    # 获取x和y的网格
    x_bins = np.linspace(0, box_size[0], bin_xy)
    y_bins = np.linspace(0, box_size[1], bin_xy)

    # 根据z范围筛选密度数据
    z_min, z_max = z_range
    z_filter = (density_map['z'] >= z_min) & (density_map['z'] <= z_max)
    
    # 提取筛选后的x和y坐标
    filtered_positions = density_map['positions'][z_filter]
    
    # 计算x和y的二维密度分布
    x_positions = filtered_positions[:, 0]
    y_positions = filtered_positions[:, 1]

    # 使用np.histogram2d来生成二维密度图
    density, _, _ = np.histogram2d(x_positions, y_positions, bins=[x_bins, y_bins])

    # 选择不同的合并方法
    if method == 'sum':
        # 按照xy平面的密度求和
        density = np.sum(density, axis=2)

    # 创建一个热图
    plt.imshow(density.T, origin='lower', extent=(0, box_size[0], 0, box_size[1]), cmap='viridis', interpolation='nearest')
    plt.colorbar(label='Density')
    plt.title(f'Density distribution in xy-plane (z={z_range[0]}-{z_range[1]} Å)')
    plt.xlabel('X (Å)')
    plt.ylabel('Y (Å)')
    plt.show()

In [453]:
file_name = "4000-cry_300k_out.xyz"
#########
file_path = os.path.join(root,file_name)

In [455]:
positions, atoms, num_atoms,box_len = read_xyz(file_path, 0)
density_map = compute_3d_density(positions, elements, box_size, bins=40)
plot_xy_density_in_z_range(density_map, box_size = boxsize, z_range=[55, 65], bin_xy=40, method='sum')

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [155]:
# 设定周期性盒子的尺寸，单位是Å
# 运行主程序，输入xyz文件路径
xyz_file = file_path  # 替换为你的xyz文件路径
# 输入你要分析的帧数（例如第2帧）
ana_frame = 0  # 替换为你想要的帧数
# 运行程序
analyze_density(xyz_file, ana_frame,boxsize,num_bins=10)


TypeError: only length-1 arrays can be converted to Python scalars